# 3.2 Incompressible Navier-Stokes equations

As in [the whetting the appetite demo](../wta/navierstokes.ipynb) we consider the unsteady Navier Stokes equations

Find $(u,p):[0,T] \to (H_{0,D}^1)^d \times L^2$, s.t.

\begin{align*}
\int_{\Omega} \partial_t u \cdot v + \int_{\Omega} \nu \nabla u \nabla v + u \cdot \nabla u v - \int_{\Omega} \operatorname{div}(v) p &= \int f v  && \forall v \in (H_{0,D}^1)^d, \\ 
- \int_{\Omega} \operatorname{div}(u) q &= 0 && \forall q \in L^2, \\
\quad u(t=0) & = u_0
\end{align*}

### Schäfer-Turek benchmark
We consider the benchmark setup of from http://www.featflow.de/en/benchmarks/cfdbenchmarking/flow/dfg_benchmark2_re100.html . The geometry is a 2D channel with a circular obstacle which is positioned (only slightly) off the center of the channel. The geometry:

![title](geometry.png) 

The viscosity is set to $\nu = 10^{-3}$.

In [3]:
# import libarries, define geometry and generate mesh
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
from netgen.webgui import Draw as DrawGeo
from math import pi

maxh  = 0.2
mesh = Mesh(unit_square.GenerateMesh(maxh=maxh))
h = specialcf.mesh_size
n = specialcf.normal

In [4]:
nu = 0.1
dt = 0.001
gammau = 0.001

In [5]:
# dirichlet="left|right|bottom|top"

In [6]:
tstart = 0.0 
t = Parameter(tstart)
u1 =  CoefficientFunction(t*t)
u_t = u1.Diff(t)
u_t(mesh(0.0))

0.0

In [7]:
u_exact = CoefficientFunction((cos(t)*(2*sin(pi*x)**2*y*(1-y)*(1-2*y)),
                          cos(t)*(-pi*sin(2*pi*x)*y**2*(1-y)**2 ) ) )
p_exact = CoefficientFunction(cos(t)*sin(pi*x)*cos(pi*y) )


In [8]:
div_sol = u_exact[0].Diff(x) + u_exact[1].Diff(y) 
#div_sol(mesh(0.4,0.4))
div_square_L2 = Integrate(div_sol**2,mesh,order=12)
print("div_square_L2 = ", div_square_L2)

div_square_L2 =  4.8598484727884536e-33


In [9]:
rhs = CoefficientFunction(( u_exact[0].Diff(t) + (u_exact[0]*u_exact[0].Diff(x) + u_exact[1]*u_exact[0].Diff(y)) 
                           - nu* ( u_exact[0].Diff(x).Diff(x) + u_exact[0].Diff(y).Diff(y) ) +  p_exact.Diff(x) ,
                           u_exact[1].Diff(t) + (u_exact[0]*u_exact[1].Diff(x) + u_exact[1]*u_exact[1].Diff(y)) 
                           - nu* ( u_exact[1].Diff(x).Diff(x) + u_exact[1].Diff(y).Diff(y) ) +  p_exact.Diff(y)
                          ) )

In [10]:
mean_pressure = Integrate(p_exact,mesh,order=12)
print("mean pressure = ", mean_pressure) 

mean pressure =  5.0306980803327406e-17


In [11]:
#DrawGeo (shape)
Draw (mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

Viscosity:

### Taylor-Hood velocity-pressure pair

We use a Taylor-Hood discretization of degree $k$. The finite element space for the (vectorial) velocity $\mathbf{u}$ and the pressure $p$ are:

\begin{align}
\mathbf{u} \in \mathbf{V} &= \{ v \in H^1(\Omega) | v|_T \in \mathcal{P}^k(T) \}^2 \\
p \in Q &= \{ q \in H^1(\Omega) | q|_T \in \mathcal{P}^{k-1}(T) \}
\end{align}

In [18]:
k = 2
V1 = H1(mesh,order=1, dirichlet="left|right|bottom|top") 
V = VectorH1(mesh,order=k, dirichlet="left|right|bottom|top")
Q = H1(mesh,order=k)
L = NumberSpace(mesh)
X = V*Q*L

The `VectorH1` is the product space of two `H1` spaces with convenience operators for identity, `grad`(`Grad`) and `div`. 

### Stokes problem for initial values
Find $\mathbf{u} \in \mathbf{V}$, $p \in Q$ so that

\begin{align}
\int_{\Omega} \nu \nabla \mathbf{u} : \nabla \mathbf{v} - \int_{\Omega} \operatorname{div}(\mathbf{v}) p &= \int \mathbf{f}  \cdot \mathbf{v}  && \forall \mathbf{v} \in \mathbf{V}, \\- \int_{\Omega} \operatorname{div}(\mathbf{u}) q &= 0 && \forall q \in Q,
\end{align}

with boundary conditions:

 * the inflow boundary data ('inlet') with mean value $1$
 
\begin{align}
u_x(0,y) &= 6 y(0.41-y)/(0.41)^2, \quad \int_{0}^{0.41} u_x(0,y) dy = 1 \\
u_y(0,y) &= 0
\end{align}
 
 * "do-nothing" boundary conditions on the ouflow ('outlet') and
 * homogeneous Dirichlet conditions on all other boundaries ('wall').

In [19]:
gfu = GridFunction(X)
(u,p,lam), (v,q,mu) = X.TnT() 

u_0 = GridFunction(X)
u_1 = GridFunction(X)
ustar = GridFunction(X)
dustar = GridFunction(X)

# Set initial values using L2-projection
t.Set(0.0*dt)

mmass = BilinearForm(X)
mmass += InnerProduct(u,v)*dx
mmass.Assemble()
invm = mmass.mat.Inverse(X.FreeDofs())

for i,ui in zip(range(2),[u_0,u_1]):
    t.Set(i*dt)
    fm = LinearForm(X)   
    fm += InnerProduct( u_exact , v) * dx
    fm.Assemble()
    ui.vec.data = invm * fm.vec

Draw(u_0.components[0] ,mesh,"u0")
Draw(u_1.components[0] ,mesh,"u1")

velocity, pressure, meanp = gfu.components
# parabolic inflow at bc=1:
#uin = CoefficientFunction((1.5*4*y*(0.41-y)/(0.41*0.41),0))
#velocity.Set(uin, definedon=mesh.Boundaries("inlet"))

#Draw(velocity,mesh,"u")
#Draw(pressure,mesh,"p")

#Draw(u_exact,mesh,"u-exact")
#Draw( p_exact,mesh,"p-exact")

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

In [26]:
beta_norm = GridFunction(V1)
beta_norm.Set(sqrt(InnerProduct(u_0.components[0],u_0.components[0] )) )
print( max(beta_norm.vec.FV().NumPy()[:]))
Draw(beta_norm ,mesh,"beta")

0.21689631212908242


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

#### inflow profile plot:

To solve the Stokes (and later the Navier-Stokes) problem, we introduce the bilinear form:

$$
  a((u,p),(v,q)) := \int_{\Omega} \nu \nabla u : \nabla v - \operatorname{div}(v) p - \operatorname{div}(u) q
$$

and directly use it to solve the Stokes problem for suitable initial values:

In [12]:
# (mu * p + lam * q)

In [13]:


a = BilinearForm(X)
stokes = (nu*InnerProduct(grad(u),grad(v))-div(u)*q-div(v)*p + mu * p + lam * q)*dx
a += stokes
a.Assemble()

f = LinearForm(X)   
f += InnerProduct(rhs , v) * dx
f.Assemble()

res = f.vec - a.mat*gfu.vec
inv_stokes = a.mat.Inverse(X.FreeDofs())
gfu.vec.data += inv_stokes * res

In [14]:
sceneu = Draw(velocity,mesh,"u")
scenep = Draw(pressure,mesh,"p")

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

## IMEX time discretization
For the time integration we consider an semi-implicit Euler method (IMEX) where the convection is treated only explicitly and the remaining part implicitly:

Find $(\mathbf{u}^{n+1},p^{n+1}) \in X = \mathbf{V} \times Q$, s.t. for all $(\mathbf{v},q) \in X = \mathbf{V} \times Q$

\begin{align}
\underbrace{m(\mathbf{u}^{n+1},\mathbf{v}) + \Delta t ~\cdot~a((\mathbf{u}^{n+1},p^{n+1}),(\mathbf{v},q))}_{ \to M^\ast} ~=~m(\mathbf{u}^{n},\mathbf{v}) - \Delta t ~\cdot~c(\mathbf{u}^{n}; \mathbf{u}^{n},\mathbf{v}) 
\end{align}

with 
\begin{align}
m(\mathbf{u},\mathbf{v}) = \int \mathbf{u} \cdot \mathbf{v}
\end{align}

and 
\begin{align}
c(\mathbf{w},\mathbf{u},\mathbf{v}) = \int \mathbf{w} \cdot \nabla \mathbf{u} \cdot \mathbf{v}
\end{align}

We prefer the incremental form (as it homogenizes the boundary conditions):
\begin{align}
m(\delta \mathbf{u}^{n+1},\mathbf{v}) + \Delta t ~\cdot~a((\delta \mathbf{u}^{n+1},\delta p^{n+1}),(\mathbf{v},q)) ~=~ \Delta t (-a((\mathbf{u}^{n},p^n),(\mathbf{v},q)) -c(\mathbf{u}^{n}; \mathbf{u}^{n},\mathbf{v}))
\end{align}

In [15]:

# matrix for implicit part of IMEX(1) scheme:
mstar = BilinearForm(X)
mstar += InnerProduct(u,v)*dx + dt*stokes
mstar.Assemble()
inv = mstar.mat.Inverse(X.FreeDofs())



Next, we want to evaluate the convection term $c(u^n;u^n,v)$ for given $u^n$, i.e. a linear form. We can do this by setting up a `LinearForm`
```
fc = LinearForm(X)
fc += (Grad(gfu) * gfu) * v * dx
```
Then, for $u^n =$`gfu` a call `fc.Assemble()` yields a vector `fc.vec`representing the linear form. 

Alternatively, we can use the semi-linear form structure of the convection form and put it into a `BilinearForm` (which indeed is able to represent non-bilinear forms as we will discuss in more detail in [unit-3.3](../unit-3.3-nonlinear/nonlinear.ipynb)). We will not need the assembly (or the storage of a sparse matrix) though:

In [16]:
conv = BilinearForm(X, nonassemble = True)
conv += (Grad(u) * u) * v * dx

Instead of assembling a `LinearForm` in every time step, we use an operator application of a `BilinearForm`, see [unit-2.11](../unit-2.11-matrixfree/matrixfree.ipynb) or the next units for more details.

The `BilinearForm`(which indeed is rather a semi-linear form) can compute the linear form at a given state `u`:

 * either by a call `conv.Apply(gfu.vec,res)` where the result ends up in the vector `res`
 * alternatively we can use the notation of an operator application `conv.mat * gfu.vec` yielding the same.

Below, we will use the latter.

In [17]:
tend = 0
gfut = GridFunction(gfu.space,multidim=0)

In [18]:
# implicit Euler/explicit Euler splitting method:
tend += 1
gfut.AddMultiDimComponent(gfu.vec)
tref = 0; cnt = 0
while t < tend-0.5*dt:
    print ("\rt=", t, end="")
    # Step 1
    ustar.vec.data = 1.5*u_1.vec 
    ustar.vec.data -= 0.5*u_0.vec
    conv.Assemble()
    
    beta_norm.Set(sqrt(InnerProduct(ustar.components[0],ustar.components[0] )) )
    beta_infty = max(beta_norm.vec.FV().NumPy()[:])
    print("beta_infty = ", beta_infty )
    
    jump_ustar = grad(ustar) -  grad(ustar).Other()
    jump_v =  grad(v) -  grad(v).Other()
    
    f = LinearForm(X)   
    f += ( (-1) * InnerProduct(rhs , v) ) * dx
    f += (  nu*InnerProduct(grad(ustar),grad(v)) ) * dx
    f += gammau * h**2 * (  sqrt((ustar*n)**2) + 0.05*beta_infty  ) * InnerProduct (jump_ustar, jump_v  ) * dx(skeleton=True)
    f.Assemble()
    res =  f.vec
    res += conv.mat * ustar.vec
    dustar.vec.data =  invm * fm.vec
    
    # Step 2 
    
    
    tref = tref + dt; cnt += 1
    
    t.Set(tref) 
    f = LinearForm(X)   
    f += InnerProduct(rhs , v) * dx
    #f += InnerProduct(rhs , v) * dx
    
    
    
    res += mmass.mat * gfu.vec
    res -= - dt * conv.mat * gfu.vec
    gfu.vec.data = inv * res    
    
    
    l2_error_vel = sqrt(Integrate( InnerProduct( gfu.components[0] - u_exact ,gfu.components[0] - u_exact  ) ,mesh ))
    l2_error_pressure = sqrt(Integrate( InnerProduct( gfu.components[1] - p_exact ,gfu.components[1] - p_exact  ) ,mesh ))
    print("L2-error=velocity = ",l2_error_vel )
    print("L2-error=pressure = ",l2_error_pressure )
    
    sceneu.Redraw()
    scenep.Redraw()
    if cnt % 50 == 0:
        gfut.AddMultiDimComponent(gfu.vec)

t= ParameterCF, val = 0
L2-error=velocity =  0.0031068755643873583
L2-error=pressure =  0.025669680280444314
t= ParameterCF, val = 0.001
L2-error=velocity =  0.0031103653745856216
L2-error=pressure =  0.025706366151775132
t= ParameterCF, val = 0.002
L2-error=velocity =  0.0031138227525547165
L2-error=pressure =  0.025739251060719955
t= ParameterCF, val = 0.003
L2-error=velocity =  0.003117234190058516
L2-error=pressure =  0.025768792456259623
t= ParameterCF, val = 0.004
L2-error=velocity =  0.0031205896977768047
L2-error=pressure =  0.025795384292233298
t= ParameterCF, val = 0.005
L2-error=velocity =  0.0031238819910073073
L2-error=pressure =  0.025819366825609803
t= ParameterCF, val = 0.006
L2-error=velocity =  0.003127105877616648
L2-error=pressure =  0.025841034769515974
t= ParameterCF, val = 0.007
L2-error=velocity =  0.0031302577926976744
L2-error=pressure =  0.02586064409436482
t= ParameterCF, val = 0.008
L2-error=velocity =  0.0031333354410941583
L2-error=pressure =  0.025878417

t= ParameterCF, val = 0.075
L2-error=velocity =  0.0032269785790071154
L2-error=pressure =  0.025991890629872533
t= ParameterCF, val = 0.076
L2-error=velocity =  0.003227453236329608
L2-error=pressure =  0.02598899912981027
t= ParameterCF, val = 0.077
L2-error=velocity =  0.00322791291435177
L2-error=pressure =  0.025986048280396055
t= ParameterCF, val = 0.078
L2-error=velocity =  0.0032283578657610755
L2-error=pressure =  0.025983038707718144
t= ParameterCF, val = 0.079
L2-error=velocity =  0.003228788335586735
L2-error=pressure =  0.02597997100782171
t= ParameterCF, val = 0.08
L2-error=velocity =  0.0032292045614997016
L2-error=pressure =  0.025976845748523682
t= ParameterCF, val = 0.081
L2-error=velocity =  0.003229606774098642
L2-error=pressure =  0.025973663471094824
t= ParameterCF, val = 0.082
L2-error=velocity =  0.0032299951971824837
L2-error=pressure =  0.025970424691829627
t= ParameterCF, val = 0.083
L2-error=velocity =  0.0032303700480105346
L2-error=pressure =  0.0259671299

L2-error=velocity =  0.003229324022751794
L2-error=pressure =  0.025565620814247614
t= ParameterCF, val = 0.161
L2-error=velocity =  0.0032290120877883845
L2-error=pressure =  0.025558631093609275
t= ParameterCF, val = 0.162
L2-error=velocity =  0.003228693999920364
L2-error=pressure =  0.025551598023598292
t= ParameterCF, val = 0.163
L2-error=velocity =  0.0032283697965189762
L2-error=pressure =  0.025544521658715
t= ParameterCF, val = 0.164
L2-error=velocity =  0.0032280395143332186
L2-error=pressure =  0.02553740205283647
t= ParameterCF, val = 0.165
L2-error=velocity =  0.0032277031895033797
L2-error=pressure =  0.025530239259232597
t= ParameterCF, val = 0.166
L2-error=velocity =  0.003227360857574181
L2-error=pressure =  0.025523033330601692
t= ParameterCF, val = 0.167
L2-error=velocity =  0.0032270125535077254
L2-error=pressure =  0.025515784319082524
t= ParameterCF, val = 0.168
L2-error=velocity =  0.0032266583116958725
L2-error=pressure =  0.025508492276282425
t= ParameterCF, va

L2-error=velocity =  0.0031775304560933875
L2-error=pressure =  0.024736211192687353
t= ParameterCF, val = 0.254
L2-error=velocity =  0.0031767521522745455
L2-error=pressure =  0.02472538549904378
t= ParameterCF, val = 0.255
L2-error=velocity =  0.0031759695956163664
L2-error=pressure =  0.024714520578449445
t= ParameterCF, val = 0.256
L2-error=velocity =  0.003175182796650248
L2-error=pressure =  0.024703616474789136
t= ParameterCF, val = 0.257
L2-error=velocity =  0.003174391765793075
L2-error=pressure =  0.02469267323204336
t= ParameterCF, val = 0.258
L2-error=velocity =  0.003173596513349064
L2-error=pressure =  0.024681690894281776
t= ParameterCF, val = 0.259
L2-error=velocity =  0.0031727970495115223
L2-error=pressure =  0.02467066950567084
t= ParameterCF, val = 0.26
L2-error=velocity =  0.0031719933843645442
L2-error=pressure =  0.02465960911047035
t= ParameterCF, val = 0.261
L2-error=velocity =  0.003171185527884724
L2-error=pressure =  0.024648509753035956
t= ParameterCF, val 

L2-error=velocity =  0.0031068796553066877
L2-error=pressure =  0.02380488045779369
t= ParameterCF, val = 0.33
L2-error=velocity =  0.0031058019354777704
L2-error=pressure =  0.023791206069347464
t= ParameterCF, val = 0.331
L2-error=velocity =  0.0031047205320300273
L2-error=pressure =  0.02377749610141856
t= ParameterCF, val = 0.332
L2-error=velocity =  0.0031036354503195217
L2-error=pressure =  0.02376375060657613
t= ParameterCF, val = 0.333
L2-error=velocity =  0.003102546695666386
L2-error=pressure =  0.02374996963751064
t= ParameterCF, val = 0.334
L2-error=velocity =  0.003101454273355441
L2-error=pressure =  0.023736153247034483
t= ParameterCF, val = 0.335
L2-error=velocity =  0.003100358188636697
L2-error=pressure =  0.02372230148808442
t= ParameterCF, val = 0.336
L2-error=velocity =  0.0030992584467258704
L2-error=pressure =  0.023708414413716013
t= ParameterCF, val = 0.337
L2-error=velocity =  0.0030981550528048846
L2-error=pressure =  0.023694492077108324
t= ParameterCF, val 

...

L2-error=velocity =  0.0025584243517142924
L2-error=pressure =  0.017550219175964377
t= ParameterCF, val = 0.668
L2-error=velocity =  0.002556321032487072
L2-error=pressure =  0.017528179375529102
t= ParameterCF, val = 0.669
L2-error=velocity =  0.0025542152698136015
L2-error=pressure =  0.0175061272356034
t= ParameterCF, val = 0.67
L2-error=velocity =  0.002552107067237768
L2-error=pressure =  0.01748406283498645
t= ParameterCF, val = 0.671
L2-error=velocity =  0.0025499964283046336
L2-error=pressure =  0.01746198625249032
t= ParameterCF, val = 0.672
L2-error=velocity =  0.002547883356560402
L2-error=pressure =  0.017439897566944945
t= ParameterCF, val = 0.673
L2-error=velocity =  0.002545767855552422
L2-error=pressure =  0.01741779685718949
t= ParameterCF, val = 0.674
L2-error=velocity =  0.002543649928829171
L2-error=pressure =  0.01739568420208257
t= ParameterCF, val = 0.675
L2-error=velocity =  0.0025415295799402437
L2-error=pressure =  0.017373559680490716
t= ParameterCF, val = 0

t= ParameterCF, val = 0.749
L2-error=velocity =  0.0023781531042082996
L2-error=pressure =  0.015708968233644757
t= ParameterCF, val = 0.75
L2-error=velocity =  0.002375861277320864
L2-error=pressure =  0.01568617881085249
t= ParameterCF, val = 0.751
L2-error=velocity =  0.0023735673005548224
L2-error=pressure =  0.015663383519067285
t= ParameterCF, val = 0.752
L2-error=velocity =  0.0023712711775149002
L2-error=pressure =  0.015640582436776185
t= ParameterCF, val = 0.753
L2-error=velocity =  0.0023689729118060773
L2-error=pressure =  0.015617775642440638
t= ParameterCF, val = 0.754
L2-error=velocity =  0.0023666725070335716
L2-error=pressure =  0.01559496321449988
t= ParameterCF, val = 0.755
L2-error=velocity =  0.0023643699668028807
L2-error=pressure =  0.015572145231366982
t= ParameterCF, val = 0.756
L2-error=velocity =  0.002362065294719698
L2-error=pressure =  0.015549321771429571
t= ParameterCF, val = 0.757
L2-error=velocity =  0.0023597584943899556
L2-error=pressure =  0.0155264

L2-error=velocity =  0.002190760178690472
L2-error=pressure =  0.013896680926849392
t= ParameterCF, val = 0.829
L2-error=velocity =  0.0021883096197512285
L2-error=pressure =  0.013873667024298751
t= ParameterCF, val = 0.83
L2-error=velocity =  0.002185857195537072
L2-error=pressure =  0.013850653339391408
t= ParameterCF, val = 0.831
L2-error=velocity =  0.002183402909635307
L2-error=pressure =  0.013827639947140797
t= ParameterCF, val = 0.832
L2-error=velocity =  0.0021809467656325504
L2-error=pressure =  0.013804626922494812
t= ParameterCF, val = 0.833
L2-error=velocity =  0.002178488767114654
L2-error=pressure =  0.013781614340335476
t= ParameterCF, val = 0.834
L2-error=velocity =  0.0021760289176667785
L2-error=pressure =  0.013758602275478139
t= ParameterCF, val = 0.835
L2-error=velocity =  0.0021735672208733226
L2-error=pressure =  0.01373559080267206
t= ParameterCF, val = 0.836
L2-error=velocity =  0.0021711036803179
L2-error=pressure =  0.013712579996597411
t= ParameterCF, val 

L2-error=velocity =  0.0020020332867006834
L2-error=pressure =  0.012176218168911897
t= ParameterCF, val = 0.904
L2-error=velocity =  0.0019994527247528047
L2-error=pressure =  0.012153423642372665
t= ParameterCF, val = 0.905
L2-error=velocity =  0.0019968705638267795
L2-error=pressure =  0.012130634739304874
t= ParameterCF, val = 0.906
L2-error=velocity =  0.0019942868074252066
L2-error=pressure =  0.012107851528212532
t= ParameterCF, val = 0.907
L2-error=velocity =  0.001991701459049157
L2-error=pressure =  0.012085074077493247
t= ParameterCF, val = 0.908
L2-error=velocity =  0.001989114522198136
L2-error=pressure =  0.012062302455432647
t= ParameterCF, val = 0.909
L2-error=velocity =  0.001986526000370142
L2-error=pressure =  0.012039536730205429
t= ParameterCF, val = 0.91
L2-error=velocity =  0.0019839358970616073
L2-error=pressure =  0.012016776969876129
t= ParameterCF, val = 0.911
L2-error=velocity =  0.0019813442157673934
L2-error=pressure =  0.011994023242396673
t= ParameterCF,

t= ParameterCF, val = 0.987
L2-error=velocity =  0.001780022575655594
L2-error=pressure =  0.010287378264223782
t= ParameterCF, val = 0.988
L2-error=velocity =  0.001777319747864376
L2-error=pressure =  0.010265283274191109
t= ParameterCF, val = 0.989
L2-error=velocity =  0.001774615609212426
L2-error=pressure =  0.010243199220147697
t= ParameterCF, val = 0.99
L2-error=velocity =  0.0017719101630471812
L2-error=pressure =  0.01022112615917569
t= ParameterCF, val = 0.991
L2-error=velocity =  0.0017692034127139938
L2-error=pressure =  0.010199064148192742
t= ParameterCF, val = 0.992
L2-error=velocity =  0.0017664953615561126
L2-error=pressure =  0.010177013243952027
t= ParameterCF, val = 0.993
L2-error=velocity =  0.001763786012914671
L2-error=pressure =  0.010154973503041327
t= ParameterCF, val = 0.994
L2-error=velocity =  0.001761075370128722
L2-error=pressure =  0.01013294498188182
t= ParameterCF, val = 0.995
L2-error=velocity =  0.0017583634365351995
L2-error=pressure =  0.0101109277

In [19]:
Draw (gfut.components[0], mesh, interpolate_multidim=True, animate=True,
      min=-0.1, max=0.1, autoscale=False, vectors = True)
Draw (gfut.components[1], mesh, interpolate_multidim=True, animate=True,
      min=-0.5, max=0.5, autoscale=False)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [20]:
maxhs  = [0.2,0.1,0.05] 
err_v =  [0.25180908428244286,0.06042844667099612 ]
err_p = [ 0.05064082177944421,0.007755257351949141  ] 
#err_v =  [0.0101407566076981,0.001747502860750249,0.00044342729502770646 ]
#err_p = [0.027880013901244633,0.010022972635385803,0.006682130004079638 ]

In [21]:
eoc = [ log(err_v[i-1]/err_v[i])/log(2) for i in range(1,len(err_v))]
print("eoc = ", eoc)

eoc =  [2.059030567990694]


In this benchmark scenario of specific interests are the forces acting on the disk (cylinder). We will discuss how to compute them next.